In [89]:
import numpy as np
import pandas as pd
import datetime
from dateutil.parser import parse

In [90]:
import json

In [91]:
with open('snowfalls.json', 'r') as infile:
    snowfalls_json = json.load(infile)

# Explore the structure of the JSON-like object holding the snowfall amounts by month.

In [92]:
len(snowfalls_json) # one dictionary

1

In [93]:
type(snowfalls_json)

dict

In [94]:
snowfalls_json.keys() # the first-level key points to a list of dictionaries

dict_keys(['snowfalls'])

In [95]:
snowfalls_json['snowfalls'][0].keys() # the third level - keys of dicts inside the list

dict_keys(['date', 'data'])

In [96]:
snowfalls_json['snowfalls'][0]['date']

'201701'

In [97]:
snowfalls_json['snowfalls'][0]['data'].keys()

dict_keys(['description', 'data'])

In [98]:
snowfalls_json['snowfalls'][0]['data']['data'].keys() # the fourth level keys in 'data'

dict_keys(['USC00210018', 'USC00210075', 'USC00210146', 'USC00210157', 'US1MNAA0054', 'USC00210252', 'USC00210287', 'US1MNMW0003', 'USC00210355', 'USC00210387', 'US1MNCS0003', 'US1MNOT0003', 'US1MNSH0019', 'US1MNKD0001', 'USC00210643', 'USC00210667', 'USC00210746', 'US1MNSH0010', 'US1MNRV0008', 'US1MNRV0005', 'USC00210809', 'USC00210855', 'US1MNIS0008', 'USC00210939', 'US1MNCW0008', 'US1MNCW0003', 'USC00210974', 'USC00210981', 'USC00210989', 'US1MNKB0003', 'US1MNHN0134', 'USC00211063', 'USC00211065', 'USC00211074', 'USC00211174', 'USC00211198', 'US1MNHS0004', 'USC00211227', 'USC00211250', 'USC00211263', 'USC00211310', 'US1MNCN0001', 'US1MNCV0001', 'USC00211374', 'USC00211422', 'US1MNCV0027', 'USC00211448', 'USC00211468', 'US1MNFM0001', 'US1MNSL0015', 'US1MNDD0011', 'USC00211630', 'USC00211691', 'US1MNBW0008', 'US1MNSL0034', 'USC00211771', 'US1MNWG0033', 'USC00211840', 'US1MNPK0013', 'US1MNSL0096', 'USC00212023', 'USC00212038', 'US1MNOT0007', 'USC00212088', 'US1MNOT0015', 'USC00212142',

In [99]:
snowfalls_json['snowfalls'][0]['data']['data']['USC00210018'].keys()  # all stations for the first month's data - fifth level

dict_keys(['station_name', 'lat', 'lon', 'state', 'county', 'elev', 'date', 'values'])

In [100]:
snowfalls_json['snowfalls'][0]['data']['data']['USC00210018']['date']

'20170101'

In [102]:
parse('20170101')

datetime.datetime(2017, 1, 1, 0, 0)

In [103]:
snowfalls_json['snowfalls'][0]['data']['data']['USC00210018']['values'].keys() # 6 levels deep

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'])

In [104]:
len(snowfalls_json['snowfalls']) # there are 11 months of data here

11

In [105]:
# Helper function to add the day key to the base date

def fix_dates(first_day_of_month, calendar_day):
    return first_day_of_month + datetime.timedelta(days=(int(calendar_day) - 1))

In [106]:
fix_dates(parse('20170101'), '15') # should be January 15, 2019

datetime.datetime(2017, 1, 15, 0, 0)

In [107]:
# snowfalls_json['snowfalls'][0]['data']['data']['USC00210018']['values']

In [108]:
def station_df(index, station):
    # We do not know if a value is 'M' for missing because there is 
    # less than 'T'race amounts of snow or simply not observed.  Therefore,
    # return np.nan (NaN)
    def convert_snow (amount):
        if (amount == 'T'):
            return 0.01
        elif (amount == 'M'):
            return np.nan
        else:
            return float(amount)
            
    
    first_day_of_month = parse(snowfalls_json['snowfalls'][index]['data']['data'][station]['date'])
    snowfall_dict = snowfalls_json['snowfalls'][index]['data']['data'][station]['values']
    
    dates = [fix_dates(first_day_of_month, date) for date in snowfall_dict.keys()]
    snow = [convert_snow(amount) for amount in snowfall_dict.values()]
    
    return pd.DataFrame({
        'station': station,
        'name': snowfalls_json['snowfalls'][index]['data']['data'][station]['station_name'],
        'county': snowfalls_json['snowfalls'][index]['data']['data'][station]['county'],
        'longitude': float( snowfalls_json['snowfalls'][index]['data']['data'][station]['lon'] ),
        'latitude': float( snowfalls_json['snowfalls'][index]['data']['data'][station]['lat'] ),
        'date': dates,
        'snowfall': snow
    })


# Compare the Stucture of the JSON object to the DataFrame we produce below
Both give all snowfall amounts for one repoting station for one month.  The DataFame is easier to work with but contains alot of repeated information.

In [109]:
snowfalls_json['snowfalls'][0]['data']['data']['USC00210018']

{'station_name': 'ADA',
 'lat': '47.30',
 'lon': '-96.52',
 'state': 'MN',
 'county': 'NORMAN',
 'elev': '907',
 'date': '20170101',
 'values': {'1': '0.0',
  '2': '2.0',
  '3': '4.2',
  '4': '0.0',
  '5': '0.0',
  '6': '0.0',
  '7': '0.0',
  '8': '0.0',
  '9': '0.5',
  '10': '1.0',
  '11': '3.5',
  '12': '0.0',
  '13': '0.0',
  '14': '0.0',
  '15': '0.0',
  '16': '0.0',
  '17': '0.0',
  '18': '0.0',
  '19': '0.0',
  '20': '0.0',
  '21': '0.0',
  '22': '0.0',
  '23': '0.0',
  '24': '0.0',
  '25': '0.0',
  '26': '0.0',
  '27': '0.0',
  '28': '0.0',
  '29': '0.0',
  '30': '0.0',
  '31': '0.0'}}

In [110]:
station_df(0,'USC00210018')

,station,name,county,longitude,latitude,date,snowfall
0,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-01,0.0
1,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-02,2.0
2,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-03,4.2
3,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-04,0.0
4,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-05,0.0
5,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-06,0.0
6,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-07,0.0
7,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-08,0.0
8,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-09,0.5
9,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-10,1.0


In [111]:
# Now grab all of the stations for one month - i.e., one position on the list:

def all_stations_one_month(index):
    
    stations = snowfalls_json['snowfalls'][index]['data']['data'].keys()
    
    list_of_df = [station_df(index, station) for station in stations]
    
    long_df = pd.concat(list_of_df)
    
    # Now - fix up the index:
    long_df.reset_index(drop=True, inplace=True)
    
    return long_df


In [112]:
date1 = all_stations_one_month(0)

In [113]:
date1.shape

(9641, 7)

In [114]:
date1.describe()

,longitude,latitude,snowfall
count,9641.000000,9641.000000,7289.000000
mean,-93.773023,45.621286,0.375334
std,1.423072,1.358493,1.018711
min,-96.870000,43.520000,0.000000
25%,-94.830000,44.660000,0.000000
50%,-93.500000,45.130000,0.000000
75%,-92.910000,46.810000,0.010000
max,-89.690000,48.840000,10.000000


In [115]:
date1

,station,name,county,longitude,latitude,date,snowfall
0,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-01,0.0
1,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-02,2.0
2,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-03,4.2
3,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-04,0.0
4,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-05,0.0
5,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-06,0.0
6,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-07,0.0
7,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-08,0.0
8,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-09,0.5
9,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-10,1.0


In [122]:
# Now piece this all together:
# Remember of json object is a global variable

def all_stations_all_months():
    
    number_of_stations = len(snowfalls_json['snowfalls'])
    
    # stations = snowfalls_json['snowfalls'][index]['data']['data'].keys()
    # remember we have one station with no data 201512 <Response [404]>
    # for the key '201512', the 'data' key maps to False
    
    list_of_df = [all_stations_one_month(index) for index in range(number_of_stations)\
                  if snowfalls_json['snowfalls'][index]['data']]
    
    long_df = pd.concat(list_of_df)
    
    # Now - fix up the index:
    long_df.reset_index(drop=True, inplace=True)
    
    return long_df


In [123]:
all_amounts_df = all_stations_all_months()

In [124]:
all_amounts_df.shape

(107058, 7)

In [126]:
all_amounts_df.describe()

,longitude,latitude,snowfall
count,107058.000000,107058.000000,78237.000000
mean,-93.746734,45.597274,0.498753
std,1.402567,1.339999,1.414293
min,-96.940000,43.520000,0.000000
25%,-94.610000,44.670000,0.000000
50%,-93.490000,45.140000,0.000000
75%,-92.890000,46.710000,0.010000
max,-89.690000,48.840000,25.000000


In [127]:
# Remove missing data
all_amounts_df.dropna(inplace=True)

In [128]:
all_amounts_df.shape

(78237, 7)

In [129]:
len(all_amounts_df['date'].unique())

301

In [130]:
len(all_amounts_df['station'].unique())

548